In [174]:
import pandas as pd
import numpy as np


# Read in data

In [175]:
df1 = pd.read_csv('7_Environmental_Taxes.csv')
df2 = pd.read_csv('14_Climate-related_Disasters_Frequency.csv')
df3 = pd.read_csv('8_Environmental_Protection_Expenditures.csv')


In [176]:
min_year = 2014
max_year = 2020
num_years = max_year + 1 - min_year

# Delete rows of non-countries

In [177]:
l_delete_countries_df2 = [
'Germany Dem Rep (former)',
'Germany Fed Rep (former)',
'Russian Federation',
'Serbia and Montenegro',
'Soviet Union (former)'
]

df2 = df2[~df2['Country'].isin(l_delete_countries_df2)]

In [178]:
l_delete_countries_df3 = [
    'Russian Federation'
]

df3 = df3[~df3['Country'].isin(l_delete_countries_df3)]

# Transform the columns if needed


In [179]:
tmp = pd.DataFrame(list(df1.Country.unique()) * num_years, columns=['Country'])
tmp.sort_values('Country', ignore_index=True, inplace=True)
tmp['Year'] = list(range(min_year, max_year + 1)) * len(df1.Country.unique())
for col in ['Environmental Taxes', 'Taxes on Energy (including fuel for transport)',
            'Taxes on Pollution', 'Taxes on Resources', 'Taxes on Transport (excluding fuel for transport)']:
    tmp[col] = np.nan
df1 = df1[df1['Unit'] == 'Percent of GDP']

for index in df1.index:
    country = df1.loc[index, 'Country']
    tax_type = df1.loc[index, 'Indicator']
    for year in [str(year) for year in range(min_year, max_year + 1)]:
        value = df1.loc[index, year]
        tmp.loc[(tmp['Country'] == country) & (tmp['Year'] == int(year)), tax_type] = value

df1 = tmp


In [180]:
tmp = pd.DataFrame(list(df2.Country.unique()) * num_years, columns=['Country'])
tmp.sort_values('Country', ignore_index=True, inplace=True)
tmp['Year'] = list(range(min_year, max_year + 1)) * len(df2.Country.unique())
for col in ['Drought', 'Extreme temperature', 'Flood', 'Landslide', 'Storm', 'Wildfire', 'Total Disasters']:
    tmp[col] = np.nan
df2['Indicator'] = df2['Indicator'].replace({'Climate related disasters frequency, Number of Disasters: Drought': 'Drought',
       'Climate related disasters frequency, Number of Disasters: Extreme temperature': 'Extreme temperature',
       'Climate related disasters frequency, Number of Disasters: Flood': 'Flood',
       'Climate related disasters frequency, Number of Disasters: Landslide': 'Landslide',
       'Climate related disasters frequency, Number of Disasters: Storm': 'Storm',
       'Climate related disasters frequency, Number of Disasters: TOTAL': 'Total Disasters',
       'Climate related disasters frequency, Number of Disasters: Wildfire': 'Wildfire'})

for index in df2.index:
    country = df2.loc[index, 'Country']
    disaster_type = df2.loc[index, 'Indicator']
    for year in [str(year) for year in range(min_year, max_year + 1)]:
        value = df2.loc[index, year]
        tmp.loc[(tmp['Country'] == country) & (tmp['Year'] == int(year)), disaster_type] = value

df2 = tmp


In [181]:
tmp = pd.DataFrame(list(df3.Country.unique()) * num_years, columns=['Country'])
tmp.sort_values('Country', ignore_index=True, inplace=True)
tmp['Year'] = list(range(min_year, max_year + 1)) * len(df3.Country.unique())
for col in ['Expenditure on biodiversity & landscape protection',
       'Expenditure on environment protection',
       'Expenditure on environmental protection n.e.c.',
       'Expenditure on environmental protection R&D',
       'Expenditure on pollution abatement',
       'Expenditure on waste management',
       'Expenditure on waste water management']:
    tmp[col] = np.nan
df3 = df3[df3['Unit'] == 'Percent of GDP']

for index in df3.index:
    country = df3.loc[index, 'Country']
    expenditure_type = df3.loc[index, 'Indicator']
    for year in [str(year) for year in range(min_year, max_year + 1)]:
        value = df3.loc[index, year]
        tmp.loc[(tmp['Country'] == country) & (tmp['Year'] == int(year)), expenditure_type] = value

df3 = tmp


# Merge the 3 dataframes

In [182]:
merged_df = pd.merge(df1, df3, on=['Country', 'Year'], how='outer')
merged_df = pd.merge(merged_df, df2, on=['Country', 'Year'], how='outer')


In [183]:
merged_df[['Drought', 'Extreme temperature', 'Flood', 'Landslide', 'Storm', 'Wildfire', 'Total Disasters']] = merged_df[['Drought', 'Extreme temperature', 'Flood', 'Landslide', 'Storm', 'Wildfire', 'Total Disasters']].fillna(0)

# Import the transformed data

In [184]:
merged_df.to_csv('data_for_dashboard2.csv')